# 1. 라이브러리 import 및 GPU 할당


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
 #GPU 사용 가능 여부를 확인하고있다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")


cuda is available


#2. 하이퍼파라미터 설정

In [6]:
# learning rate : gradient의 방향으로 얼마나 빠르게 이동할 것인지 결정
# batch size : 전체 학습 데이터를 등분하는 크기
# num classes : y(class)의 개수
# epoch : 반복 학습 횟수

learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

# 3. training 및 test 데이터셋 불러오기

In [4]:
train_set = torchvision.datasets.MNIST( #torchvision이 제공하는 데이터셋
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([ #numpy 이미지에서 torch이미지로 변경한다.(축변환)
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

#4. 학습용 데이터 준비하기

In [5]:
#데이터로더는 샘플들을 미니배치로 전달하고, 메 에포크마다 데이터를 다시 섞어 과적합을 방지한다.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

#enumerate 함수는 기본적으로 index와 원소로 이루어진 tuple을 만들어준다.
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

# 5. 모델 정의

In [8]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() # super class로 지금 작성하고 있는 클래스를 초기화

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 입력채널 수 = 1, 출력 채널 수 = 10, kernel size = 5, stribe = 1, ? = 0 인 convolution layer이다
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # dropout 비율이 0.25인 droupout layer. 오버피팅을 방지하기 위한
        self.mp = nn.MaxPool2d(2) # 합성곱 연산. 필터크기 2*2
        self.fc1 = nn.Linear(320,100) #입력 차원 320 출력차원 100의 linear layer
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) #activate 함수가 relu로 conv - maxpool - relu 의 순서
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) #데이터를 1차원 형태로 변환한다.
        x = self.fc1(x) #fully connected layer로 classification을 위해 추가
        x = self.fc2(x) 
        return F.log_softmax(x) 

# 6. loss 함수 및 optimizer 정의

In [9]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device) #loss함수는 cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) #optimization 함수는 adam

# 7. training

In [11]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # 모든 model의 gradient 값을 0으로 설정
        # 초기화 이유 : 기울기를 초기화해야만 새로운 가중치 편향에 대해 기울기를 구할 수 있다
        hypothesis = model(data)
        cost = criterion(hypothesis, target) # 모델의 결과와 target를 비고해 loss를 구한다
        cost.backward() # 역전파 시행
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1] cost = 0.0389175154
[Epoch:    2] cost = 0.0326781794
[Epoch:    3] cost = 0.0284960065
[Epoch:    4] cost = 0.0243383907
[Epoch:    5] cost = 0.0211132523


#8. validation

In [12]:
model.eval()
with torch.no_grad(): # gradient를 계산해주는 engine을 비활성화시켜 필요한 메모리를 줄이고 속도를 증가시킨다.
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.8 %
